In [4]:
#### ------Data Sampling------####
from importlib.metadata import version 
import tiktoken 

tokenizer = tiktoken.get_encoding("gpt2") 
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()
enc_text = tokenizer.encode(text)
print("number of token: ", len(enc_text))
print(enc_text[:50])

number of token:  5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]


In [5]:
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print("x: ", x)
print("y: ", y)


x:  [290, 4920, 2241, 287]
y:  [4920, 2241, 287, 257]


In [6]:
### ------Data Loader------ ###
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset): 
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids  = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, 
                          shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last, 
                            num_workers=num_workers)
    return dataloader
    


In [7]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch) 

third_batch = next(data_iter)
print(third_batch) 

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


In [8]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464],
        [1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899]])]


In [9]:
### -----Embedding------ ###
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3 
torch.manual_seed(123) 
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight) 
print("\n")
print(embedding_layer(input_ids)) 

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [10]:
### ------Bigger Embedding------ ###
vocab_size = 50257 
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim) 

max_length = 4 
dataloader = create_dataloader_v1(raw_text, batch_size=8, 
                                  max_length=max_length,    
                                  stride=max_length, shuffle=False) 
data_iter = iter(dataloader) 
inputs, targets = next(data_iter) 
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print("just token_embeddings: ", token_embeddings.shape) 

## GPT-3 uses absolute posisition embedding 
## This is also the case here
context_length = max_length
# Each location has its own position vector
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim) 
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print("pos_embeddings: ", pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings 
print(input_embeddings.shape)



Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
just token_embeddings:  torch.Size([8, 4, 256])
pos_embeddings:  torch.Size([4, 256])
torch.Size([8, 4, 256])
